# CUDA exercises

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams
import itertools
%matplotlib inline

from IPython.display import display

In [ ]:
def csv_parse(path, header=False, sep=";"):
    if header:
        df = pd.read_csv(path, sep=sep, names=header)
    else:
        df = pd.read_csv(path, sep=sep)
    #display(df.head())
    return df



In [ ]:
def calcGB_Stride(df):
    return 3*8* df["N"]/df["k"] /1e9

def calcGB_Offset(df):
    return 3*8* (df["N"]-df["k"])/1e9

def transform_to_ms(df, cols):
    return df[cols]*1e3


# Ex5.2 FD

In [ ]:
df = csv_parse("fd_data_ph.csv", header=False)
#df[times_cols] = transform_to_ms(df, times_cols) # transform to ms
display(df)

In [ ]:
cols = df.columns
times_cols = list(filter(lambda x: x not in ["N", "k"], cols))

cols, times_cols

In [ ]:
cpu_cols = list(filter(lambda x: "cpu" in x, times_cols))
gpu_cols = list(filter(lambda x: "gpu" in x, times_cols))

cpu_cols, gpu_cols

In [ ]:
labels = list(map(lambda x: x.split("_", 1)[-1], cpu_cols+gpu_cols)) + ["times_cg"]
labels

In [ ]:
reldur_cols = []
for label in labels[:-1]:
    ref = "times_cg"
    df["relative_duration_"+label] = df["times_"+label]/df[ref]
    reldur_cols.append("relative_duration_"+label)
df

In [ ]:
def print_styling():

    plt.rc('figure', figsize=(14,8)) 
    SMALL_SIZE = 15
    MEDIUM_SIZE = 18
    BIGGER_SIZE = 26

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes

    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
    
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
fig, axs = plt.subplots(ncols=2)

logx, logy = True, True
if (logx and logy):
    plot_format = "loglog"
if (logx and not logy):
    plot_format = "logx"
if (not logx and logy):
    plot_format = "logy"
else:
    plot_format = "linear"

print_styling();
x = "N"
df.plot(x=x, y=cpu_cols, logx=logx, logy=logy, marker="o", linestyle="--", ax=axs[0]);
df.plot(x=x, y=gpu_cols, logx=logx, logy=logy, marker="x", linestyle="-", ax=axs[0]);
df.plot(x=x, y="times_cg", logx=logx, logy=logy, linestyle="--", ax=axs[0]);


Nmin = 1e5
df.plot(x=x, y=reldur_cols, logx=logx, logy=logy, marker="o", linestyle="--", ax=axs[1]);

fig.suptitle("Comparison")
plt.tight_layout()
axs[0].set_ylabel("runtime [s]")
axs[1].set_title("Runtimes")
axs[1].set_title("relative to CG")

for ax in axs.ravel():
    ax.legend(labels)
    ax.grid()

fig.savefig("fd_comp.pdf", bbox_inches="tight", pad_inches=0.2)